In [14]:
import sqlite3
import pandas as pd
import os
from tabulate import tabulate

In [21]:
print(df.columns)

Index(['Book Name', 'Author Name', 'Publish Year', 'Rating', 'Genre'], dtype='object')


In [16]:
# Remove extra spaces from column names
df.columns = df.columns.str.strip()
print(df.columns)  # Verify the column names

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', 'num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')


In [29]:
# Correct file path to your Excel file
file_path = "BooksDataSet.xlsx"

# Verify the file path
if not os.path.isfile(file_path):
    raise FileNotFoundError(f"The file {file_path} does not exist.")

# Load the dataset into a DataFrame with error handling
try:
    df = pd.read_excel(file_path, engine='openpyxl')  # Use 'openpyxl' engine for .xlsx files
except pd.errors.ParserError as e:
    print(f"Error reading the Excel file: {e}")
    exit(1)

# Remove any leading or trailing spaces from column names
df.columns = df.columns.str.strip()

# Standardize column names to avoid spaces and case issues
df.columns = ['book_name', 'author_name', 'publish_year', 'rating', 'genre']

# Create a connection to the SQLite database
conn = sqlite3.connect('books.db')
cursor = conn.cursor()

# Drop the existing table if it exists
cursor.execute("DROP TABLE IF EXISTS books")

# Create the table with the standardized column names
cursor.execute('''
    CREATE TABLE books (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        book_name TEXT,
        author_name TEXT,
        publish_year TEXT,
        rating REAL,
        genre TEXT
    )
''')

# Insert data into the table
df.to_sql('books', conn, if_exists='replace', index=False)

# Debug: Check the table schema
def print_table_schema():
    cursor.execute("PRAGMA table_info(books);")
    schema = cursor.fetchall()

print_table_schema()

# Function to display available options
def display_options():
    cursor.execute("SELECT DISTINCT genre FROM books")
    genres = cursor.fetchall()
    genres = sorted([genre[0] for genre in genres if genre])

    cursor.execute("SELECT MIN(rating), MAX(rating) FROM books")
    min_rating, max_rating = cursor.fetchone()

    return genres

# Function to query books based on user preferences
def get_books_by_preferences():
    genres = display_options()

    # Display genre options again for clarity
    print("\nGenre options:")
    for i, genre in enumerate(genres, start=1):
        print(f" {i}. {genre}")

    try:
        genre_choice = int(input("\nEnter the number corresponding to your genre preference or 0 to skip: ").strip())
        genre = genres[genre_choice - 1] if genre_choice != 0 else None
    except (IndexError, ValueError):
        genre = None

    print("\nRating range:")
    print(f" - Minimum rating: 0")
    print(f" - Maximum rating: {max_rating}")
    min_rating = input("Do you have a minimum rating preference (out of 5)? (Enter rating or press enter to skip): ").strip()

    query = "SELECT book_name, author_name, publish_year, rating, genre FROM books WHERE 1=1"
    params = []

    if genre:
        query += " AND genre = ?"
        params.append(genre)

    if min_rating:
        query += " AND rating >= ?"
        params.append(float(min_rating))

    try:
        cursor.execute(query, params)
        results = cursor.fetchall()

        if results:
            headers = ['Book Name', 'Author Name', 'Publish Year', 'Rating', 'Genre']
            print("\nBooks matching your criteria:\n")
            print(tabulate(results, headers=headers, tablefmt='grid'))
        else:
            print("No books found matching your criteria.")
    except sqlite3.OperationalError as e:
        print(f"SQL Error: {e}")

# Prompt user for preferences and get matching books
get_books_by_preferences()

# Close the connection
conn.close()


Genre options:
 1. Adventure
 2. Autobiographical
 3. Autobiography
 4. Biography
 5. Classic
 6. Coming-of-Age
 7. Contemporary
 8. Drama
 9. Dystopian
 10. Fantasy
 11. Fiction
 12. Historical
 13. Historical Fiction
 14. Horror
 15. Magical Realism
 16. Modernist
 17. Mystery
 18. Philosophical
 19. Political Satire
 20. Romance
 21. Satire
 22. Science Fiction
 23. Self-Help
 24. Thriller
 25. Young Adult

Enter the number corresponding to your genre preference or 0 to skip: 23
Do you have a minimum rating preference (out of 5)? (Enter rating or press enter to skip): 3

Books matching your criteria:

+-------------+-------------------+----------------+----------+-----------+
| Book Name   | Author Name       |   Publish Year |   Rating | Genre     |
+=============+===================+================+==========+===========+
| Big Magic   | Elizabeth Gilbert |           2015 |      4.3 | Self-Help |
+-------------+-------------------+----------------+----------+-----------+
